## Import packages

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Define functions

In [ ]:
def periodic_uniax(period, c, z, amp, gamma):
    omega = (2 * np.pi) / period
    arg = omega / (2*c)
    exp_1 = np.exp(-z * np.sqrt(arg))
    exp_2 = np.exp(-1j*z*np.sqrt(arg))
    phasor = (gamma * amp) + ((1-gamma)*amp*exp_1*exp_2)
    return phasor

def periodic_fit_lst_sq(time, signal, period):
    num_times = len(time)
    num_per = len(period)

    omega = (2*np.pi)/period
    
    G = np.zeros((num_times,2*num_per))
    G[:,0] = np.ndarray.flatten(np.cos(omega * time))
    G[:,1] = np.ndarray.flatten(-np.sin(omega * time))
    
    phasor = np.linalg.solve(np.transpose(G) @ G, np.transpose(G) @ signal)
    return phasor

## Specify directories / input files

In [ ]:
input_file = './mbari_low_freq.i'
exe_file = './moose'
results_folder = './out_files'
results_file = 'mbari.csv'
file_prefix = 'mbari_depth_pp_'

## Model run

In [ ]:
run_command = exe_file+' '+ '-i'+' '+input_file+' '
os.system(run_command)

## Specify hydromechanical properties

In [ ]:
% Fluid Properties
mu = 1.26e-3;
Kf = 2e9;

% Seafloor Sediment Elastic Properties
Kd = 3.4e8;
G = 2.2e8;
biot = 0.9;
eta = 0.25

% Derived Elastic Properties
M_inv = (eta/Kf) + (((1-biot)*(biot-eta))/Kd);
M = 1/M_inv;
Ku = Kd + (biot^2 * M);
nu = ((3*Kd)-(2*G))/(2*((3*Kd)+G));
nu_u = ((3*Ku)-(2*G))/(2*((3*Ku)+G));
B = (3*(nu_u-nu)) / (biot * (1-(2*nu)) * (1+nu_u));
gamma = (B*(1+nu_u))/(3*(1-nu_u));

% Rock Hydraulic Properties
k = 4e-14;
c = k/(mu*M_inv);